<a href="https://colab.research.google.com/github/baizhankyzy/female-directors/blob/main/Project_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdflib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 9.1 MB/s eta 0:00:00


RDF GRaph

In [ ]:
from rdflib import ConjunctiveGraph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS
from urllib.parse import quote
import requests
from google.colab import files

# Fetch the JSON data
url = "https://raw.githubusercontent.com/baizhankyzy/female-directors/refs/heads/main/dataset_final.json"
response = requests.get(url)
data = response.json()['results']['bindings']

# Initialize the graph
g = ConjunctiveGraph()

# Define namespaces
wd = Namespace("http://www.wikidata.org/entity/")
wdt = Namespace("http://www.wikidata.org/prop/direct/")
wikidata = Namespace("http://www.wikidata.org/")

# Add data to the graph
for item in data:
    film = URIRef(item['film']['value'])
    director = URIRef(item['director']['value'])
    release_year = Literal(item['releaseYear']['value'], datatype=RDFS.Literal)
    film_label = Literal(item['filmLabel']['value'], lang='en')
    director_label = Literal(item['directorLabel']['value'], lang='en')

    genre_label = item.get('genreLabel', {}).get('value', "Unknown Genre")
    genre_uri = URIRef(f"http://www.wikidata.org/genre/{quote(genre_label)}")

    country_label = item.get('countryLabel', {}).get('value', "Unknown Country")
    country_uri = URIRef(f"http://www.wikidata.org/country/{quote(country_label)}")

    # Avoid blank nodes by always ensuring explicit URIs or Literals
    if isinstance(film, URIRef) and isinstance(director, URIRef):
        # Add RDF triples
        g.add((film, wdt.P31, wd.Q11424))  # Film type
        g.add((film, RDFS.label, film_label))
        g.add((film, wdt.P57, director))
        g.add((film, wdt.P577, release_year))
        g.add((film, wdt.P136, genre_uri))
        g.add((film, wdt.P19, country_uri))
        g.add((director, RDFS.label, director_label))
        g.add((genre_uri, RDFS.label, Literal(genre_label, lang='en')))
        g.add((country_uri, RDFS.label, Literal(country_label, lang='en')))

# Serialize the graph
output_file = "director_graph.nq"
g.serialize(output_file, format="nquads")

# Download the fixed file
files.download(output_file)
print(f"Data saved as {output_file} and ready for download.")

<ipython-input-3-cf22387c4fd7>:13: DeprecationWarning: ConjunctiveGraph is deprecated, use Dataset instead.
  g = ConjunctiveGraph()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data saved as director_graph.nq and ready for download.


# Loading the graph

In [3]:
from rdflib import ConjunctiveGraph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS
from urllib.parse import quote
import requests
from google.colab import files

g = ConjunctiveGraph()
g.parse("director_graph.nq", format="nquads")

<ipython-input-3-a9c862614dd6>:7: DeprecationWarning: ConjunctiveGraph is deprecated, use Dataset instead.
  g = ConjunctiveGraph()


FileNotFoundError: [Errno 2] No such file or directory: '/director_graph.nq'

Download graph from github to Google Colab

In [4]:
from rdflib import ConjunctiveGraph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS
import requests

# GitHub raw URL for the .nq file
github_raw_url = "https://raw.githubusercontent.com/baizhankyzy/female-directors/refs/heads/main/director_graph.nq"

# Download the nquads file from GitHub
nquads_file = "director_graph.nq"
response = requests.get(github_raw_url)

if response.status_code == 200:
    with open(nquads_file, 'wb') as file:
        file.write(response.content)
    print(f"File {nquads_file} downloaded successfully.")
else:
    print(f"Failed to download file: {response.status_code}")
    response.raise_for_status()

# Load the graph
g = ConjunctiveGraph()
try:
    g.parse(nquads_file, format="nquads")
    print("Graph loaded successfully.")
except Exception as e:
    print(f"Error loading graph: {e}")

# Now you can query the graph


File director_graph.nq downloaded successfully.


<ipython-input-4-b724055496a0>:21: DeprecationWarning: ConjunctiveGraph is deprecated, use Dataset instead.
  g = ConjunctiveGraph()


Graph loaded successfully.


## Research questions
### How has the number of women directors in feature films changed over the last 30 years, including fluctuations during specific periods and potential external factors such as societal events or industry shifts?




In [5]:
# Iterate through each year from 1994 to 2024 and count films
print("Year\tNumber of Films")
for year in range(1994, 2025):
    # Construct the query dynamically for each year
    query = f"""
    SELECT (COUNT(?film) AS ?film_count)
    WHERE {{
      ?film <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q11424> .
      ?film <http://www.wikidata.org/prop/direct/P577> "{year}"^^<http://www.w3.org/2000/01/rdf-schema#Literal> .
    }}
    """

    # Execute the query
    results = g.query(query)

    # Print the result for the current year
    for row in results:
        print(f"{year}\t{row['film_count']}")

Year	Number of Films
1994	224
1995	247
1996	242
1997	255
1998	292
1999	305
2000	313
2001	364
2002	371
2003	389
2004	438
2005	502
2006	553
2007	586
2008	620
2009	697
2010	614
2011	707
2012	773
2013	827
2014	824
2015	844
2016	1448
2017	897
2018	886
2019	865
2020	627
2021	712
2022	788
2023	706
2024	468


## **2nd research question**



```
# Which countries have the highest and lowest numbers of women directors in feature films?
```

